# Codice 2D per Monodominio

Script che implementa il sistema Monodominio

$$
\begin{cases}
\chi C_m \dfrac{dv}{dt} - \text{div} (\sigma \nabla u) + \chi I(v,w) = I_{app} \\
\dfrac{dw}{dt} = Gf(v,w) \\
\mathbf{n} \cdot (\sigma \nabla v) = 0
\end{cases}
$$

### Importiamo la libreria

Utilizziamo Firedrake https://www.firedrakeproject.org/ libreria _open-source_ per la risoluzione di Equazioni alle Derivate Parziali tramite metodi agli Elementi Finiti.

In [ ]:
from firedrake import *
from numpy import linspace

### Modello ionico

Risolviamo in modo disaccoppiato il modello ionico dall'equazione Monodominio per la propagazione del segnale elettrico.

Implementiamo il modello di FitzHugh-Nagumo.
La dinamica delle variabili di gating e' descritta dalla seguente Equazione Differenziale Ordinaria:

$$ \dfrac{dw}{dt} = \eta v - \gamma w $$

La corrente ionica $I(v,w)$ che viene poi considerata nella equazione Monodominio, è data da
$$
I(v,w) = - b v (v-c) (\delta -v ) + \beta w
$$
dove $\gamma, \eta,  b, c, \beta$ e $\delta$ sono costanti date.

In [ ]:
# ----- modello ionico di FitzHugh Nagumo
#
gamma = 0.025;
eta   = 0.1;
b     = 5;
c     = 0.1;
beta  = 1;
delta = 1;

# [equazioni caratteristiche del modello]
def Gf(v, w):
    return eta * v - gamma * w
def I(v, w):

### Generiamo la mesh

Definiamo alcuni parametri che serviranno per la simulazione, come il passo temporale $dt$, il tempo iniziale, finale e le condizioni iniziali e il tensore di conduttivita'

In [ ]:
dt = 0.05     # passo temporale
t  = 0.0 
Tf = 10.0     # tempo finale

# ----- costanti del problema
chi   = Constant(1.0)
cm    = Constant(1.0)
ic    = Constant(0.0)
sigma = Constant( ( (2e-3, 0), (0, 1.3514e-3) ) )

Generiamo la mesh. Firedrake permette di caricare mesh in formati differenti o di accedere a quelle presenti nella libreria standard (https://www.firedrakeproject.org/_modules/firedrake/utility_meshes.html)

Consideriamo per semplicita' una mesh quadrata 2D unitaria $[0,1] \times [0,1]$.

In [ ]:
n = 20
# mesh = RectangleMesh(nx, ny, Lx, Ly)
mehs = UnitSquareMesh(n, n)
x, y = SpatialCoordinate(mesh)

Plottiamo la mesh

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
triplot(mesh, axes=axes)

Definiamo la regione di spazio dove applicheremo lo stimolo esterno.

Consideriamo uno stimolo circolare di raggio $r$ applicato nell'angolo in basso a sinistra della mesh, con intensita' $I_a$, applicato per un tempo $t_a$:

In [ ]:
ta = 1.0     # tempo di stimolo
Ia = 20      # intensita' corrente applicata
r  = 0.1     # raggio

### Spazio degli elementi finiti

Definiamo lo spazio degli elementi finiti per il problema Monodominio e le relative funzioni.

Lista di tutti gli spazi implementati: https://www.firedrakeproject.org/variational-problems.html#supported-finite-elements

In [ ]:
# ----- load functional spaces and functions
V = FunctionSpace(mesh, "CG", 1)
w_ = Function(V, name="Gating old")
w = Function(V, name="Gating new")
u_ = Function(V, name="Potential old")
u = Function(V, name="Potential new")
v = TestFunction(V)

Imponiamo le condizioni iniziali alle f
unzioni considerate.

NB. Di default Firedrake crea f
unzioni nulle.

In [ ]:
w_.assign(ic)
w.assign(ic)
u_.assign(ic)
u.assign(ic)

Risolviamo, per ogni passo temporale, prima l'equazione per le variabili di gating, poi
il problema Monodominio

In [ ]:
# ----- time loop
Tf = 10.0
for t in ProgressBar("Time step").iter(linspace(0.0, Tf, int(Tf/dt))):
    # ----- definisco e risolvo l'equazione per la gating
    G = (inner((w - w_)/dt, v)
        - inner( Gf(u_, w_), v ) ) * dx
    solve(G == 0, w)
    w_.assign(w)
    
    # ----- definisco e risolvo l'equazione Monodominio
    Iapp = interpolate( Ia * le( sqrt(x*x + y*y), r ) * le(t, ta) , V)
    F = ( chi * cm * inner( (u - u_)/dt, v )
        + inner( sigma * grad(u) , grad(v) )
        + chi * inner( I(u_, w_), v)
        - inner(Iapp, v) ) * dx
    
    solve(F == 0, u)
    
    u_.assign(u)

Plotto la soluzione all'ultimo passo temporale

In [ ]:
fig, axes = plt.subplots()
collection = tripcolor(u, axes=axes, cmap='coolwarm')
fig.colorbar(collection);

Firedrake permette all'utente di scegliere e impostare i risolutori lineari/nonlineari ed
eventuali precondizionatori dalla libreria per il calcolo parallelo PETSc (https://petsc.org/release/overview/)

In [ ]:
solve(F == 0, u, solver_parameters={'ksp_type': 'cg',
                            'pc_type': 'none',
                            'ksp_monitor': None,
                            'ksp_monitor_singular_value': None})